In [4]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import pickle
import time
import datetime

#Extracts the following values from each IMDB page in "With_IMDB.p".
#Stores them in a pickle'd DataFrame called "IMDB_df".
#Every name in "Cast".
#Metacritic critic score.
#Director: Done
#Release date: Done
#Budget
movielibrary = pickle.load(open("With_IMDB.p", "rb"))
movies_to_skip = pickle.load(open("Movies to skip.p", "rb"))
        

In [5]:


movie_data = []
for movie in list(movielibrary.keys()):
    print(movie)
    if movie not in movies_to_skip:
        this_movie = []
        if 'IMDB Page' in movielibrary[movie]:
            imdb = BeautifulSoup(movielibrary[movie]['IMDB Page'], "lxml")
            try:
                release_date = imdb.find(title="See more release dates").text.split(" ")
                day = release_date[0]
                if len(day)==1:
                    day = "0"+day
                month = release_date[1]
                year = release_date[2]
                release_day=datetime.datetime.strptime(month+day+year, "%B%d%Y")
            except:
                release_day = None
                movies_to_skip.add(movie)

            try:
                metacritic_critics = imdb.find(class_="titleReviewBarItem").findNext().findNext().text
                if " " not in metacritic_critics:
                    score = int(metacritic_critics.strip())
                else:
                    score = None
            except:
                score = None
                

            try:
                director = imdb.find(class_="credit_summary_item").find("a").text
            except: 
                director = None
                
            try:
                budget = imdb.find(text="Budget:").findParent().findParent()
                budgetNum = budget.text
            except:
                budgetNum = None
                

            try:
                cast = []
                table = imdb.find("table", class_= "cast_list")
                for t in table.find_all("tr")[1:]:
                    name =  t.find_all("td")[1].text
                    cast.append(name)
            except:
                cast = None
                
            movie_data.append([movie, release_day, score, director, cast, budgetNum])
            movielibrary[movie]=None
        else:
            print(movie)
            movies_to_skip.add(movie)

Avengers: Endgame
The Lion King
Star Wars: Episode IX - The Rise of Skywalker
Frozen II
Toy Story 4
Captain Marvel
Spider-Man: Far from Home
Aladdin
Joker
Jumanji: The Next Level
It Chapter Two
Us
Fast & Furious Presents: Hobbs & Shaw
John Wick: Chapter 3 - Parabellum
Knives Out
How to Train Your Dragon: The Hidden World
1917
The Secret Life of Pets 2
Pokémon Detective Pikachu
Once Upon a Time... In Hollywood
Shazam!
Ford v Ferrari
Dumbo
Maleficent: Mistress of Evil
Glass
Godzilla: King of the Monsters
The Upside
Little Women
The LEGO Movie 2: The Second Part
Hustlers
The Addams Family
Downton Abbey
Rocketman
Alita: Battle Angel
Good Boys
Men in Black: International
Annabelle Comes Home
Yesterday
Tyler Perry's a Madea Family Funeral
Zombieland: Double Tap
Angel Has Fallen
Scary Stories to Tell in the Dark
Spies in Disguise
X-Men: Dark Phoenix
Terminator: Dark Fate
A Beautiful Day in the Neighborhood
Abominable
Dora and the Lost City of Gold
Escape Room
Midway
The Curse of La Llorona
Pe

In [ ]:
imdb_features = pd.DataFrame(data=movie_data, columns = ["index_Name", "Release Date", "Metacritic Critic Score", "Director", "Cast", "IMDB's Budget"])
pickle.dump(imdb_features, open("IMDB_df.p", "wb"))